In [127]:
# initial imports
import pandas as pd
import numpy as np
from numpy import percentile
import random
from sqlalchemy import create_engine
import hvplot.pandas
from dotenv import get_key

In [128]:
psqgl_credential = get_key('.env', 'CREDENTIAL')
con_str = "postgresql://" + psqgl_credential + "@localhost:5432/postgres"
pgsql_engine = create_engine(con_str)
db_connection = pgsql_engine.connect()

In [129]:
sql_query = """select card_holder_id, transaction_amount, transaction_date
from view_fraud_transactions"""
#group by card_holder_id, transaction_amount, transaction_date"""
df_test = pd.read_sql(sql_query, db_connection)#.dropna()
df_test.head()

,card_holder_id,transaction_amount,transaction_date
0,13,6.22,2018-01-01 21:35:10
1,13,3.83,2018-01-01 21:43:12
2,10,9.61,2018-01-01 22:41:21
3,4,19.03,2018-01-01 23:13:30
4,18,2.95,2018-01-01 23:15:10


In [130]:
def get_quartiles(df_iqr):
    """function to identify outliers based on quartiles"""
    df_quartile = pd.DataFrame(columns=['card_holder_id', 'q25', 'q75', 'iqr', 'cut_off', 'lower', 'upper']) #- store quartiles
    for col in df_iqr:
        if col !='transaction_date':
            try:
                q25, q75 = percentile(df_iqr[col].dropna(), 25), percentile(df_iqr[col].dropna(), 75)
                cut_off = (q75 - q25)* 1.5
                lower = q25 - (q75 - q25)* 1.5
                upper = q75 + (q75 - q25)* 1.5
                df_quartile = df_quartile.append({'card_holder_id':df_iqr[col].name, 'q25':q25 , 'q75': q75, 'iqr':q75 - q25, 
                                                 'cut-off': cut_off, 'lower': lower,
                                                 'upper': upper}, ignore_index=True)
                for amount in df_iqr[col]:
                    if amount >= upper: #as the lower threshold is below 0 and we dont have -ve transactions, ignoring it in the condition.
                        df_iqr[col].replace(amount, np.nan, inplace=True) 
                        #anything outside the quantiles replaced with np.nan(outliers removed and added to outliers df)
                        
            except:
                print(col)
    df_quartile = df_quartile.set_index('card_holder_id').dropna(axis=1) # we could use this data frame for viewing quartile info
    return (df_iqr)

In [131]:
df_outliers_removed= get_quartiles(df_test)
df_outliers_removed= df1.sort_values('card_holder_id')
df_outliers_removed.head()

,card_holder_id,transaction_amount,transaction_date
3164,1,2.99,2018-11-25 01:27:57
667,1,3.63,2018-03-09 00:31:16
3200,1,3.23,2018-11-29 04:05:12
2984,1,10.83,2018-11-05 22:30:35
2999,1,10.32,2018-11-07 12:48:48


In [132]:
df_outliers_removed.hvplot.box(by='card_holder_id', y='transaction_amount', title="Outlier Transactions Removed")

:BoxWhisker   [card_holder_id]   (transaction_amount)

In [133]:
def outlier_std(df_std):
    """function to identify outliers based on standard deviation"""
    df_outlier_std = df_std
    df_std = df_outlier_std.std()
    df_mean = df_outlier_std.mean()
    
#     Define cutof points - considering 2 standard deviation, or accounting for  95% of all transactions into consideration. <br>
#     primary reason is that we noticed note more than a handful intermediary amount transactions bt'n 50 to 1000<br>
#     for this example lets consider 2 times std. we can prompt the user to define the sigma/std levels and accordingly adjust. 
    
    std_cut_off = df_std*2
    std_lower, std_upper = df_mean - std_cut_off , df_mean + std_cut_off
    df_outlier = pd.DataFrame(columns=range(1, 26)) # -- This dataframe is to capture the removed outliers 
    outliers= []     
    for col in df_outlier_std:
        if col !='transaction_date':
            for amount in df_outlier_std[col]:
                if amount > std_upper[col]:
                    outliers.append(amount)
                    df_outlier_std[col].replace(amount, np.nan, inplace=True) 
                    #anything outside the 1 std replaced with np.nan(outliers than 1 std).
                else:
                    outliers.append(np.nan)
    df_outlier[col] = outliers 
    return df_outlier_std

In [134]:
df_outlier_removed_std = outlier_std(df_test)
df_outlier_removed_std= df_outlier_removed_std.sort_values('card_holder_id')
df_outlier_removed_std.head()

C:\Users\TinkerBell\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """


,card_holder_id,transaction_amount,transaction_date
3164,1,2.99,2018-11-25 01:27:57
274,1,10.24,2018-01-29 06:32:49
2784,1,5.13,2018-10-16 04:28:49
2768,1,11.87,2018-10-13 17:11:10
1561,1,1.98,2018-06-08 13:48:44


In [135]:
df_outlier_removed_std.hvplot.box(by='card_holder_id', y='transaction_amount', title="Outlier Transactions Removed")

:BoxWhisker   [card_holder_id]   (transaction_amount)